In [1]:
%matplotlib inline
import os
import shutil
import sys
import numpy as np
import pandas as pd
import mxnet as mx
from mxnet import gluon
from mxnet import image
from mxnet import nd
from mxnet import init
from mxnet import autograd
from mxnet.gluon.data import vision
from mxnet.gluon import nn
from mxnet.gluon.model_zoo import vision as models
from PIL import Image
from time import time
from tqdm import tqdm
import h5py
from sklearn.model_selection import train_test_split

In [2]:
def get_features(net, data):
    features = []
    labels = []

    for X, y in tqdm(data):
        feature = net.features(X.as_in_context(ctx))
        features.append(feature.asnumpy())
        labels.append(y.asnumpy())
    
    features = np.concatenate(features, axis=0)
    labels = np.concatenate(labels, axis=0)
    return features, labels

In [3]:
for doc in tqdm(os.listdir('/home/samael/kaggle/dogs/data2/train')):
    if not os.path.exists(os.path.join('/home/samael/kaggle/dogs/data3', doc)):
        os.makedirs(os.path.join('/home/samael/kaggle/dogs/data3', doc))
    for img in os.listdir(os.path.join('/home/samael/kaggle/dogs/data2/train', doc)):
        shutil.copy(os.path.join('/home/samael/kaggle/dogs/data2/train', doc, img), os.path.join('/home/samael/kaggle/dogs/data3', doc))

100%|██████████| 120/120 [00:14<00:00,  8.46it/s]


In [4]:
for doc in tqdm(os.listdir('/home/samael/kaggle/dogs/data2/train2')):
    if not os.path.exists(os.path.join('/home/samael/kaggle/dogs/data3', doc)):
        os.makedirs(os.path.join('/home/samael/kaggle/dogs/data3', doc))
    for img in os.listdir(os.path.join('/home/samael/kaggle/dogs/data2/train2', doc)):
        shutil.copy(os.path.join('/home/samael/kaggle/dogs/data2/train2', doc, img), os.path.join('/home/samael/kaggle/dogs/data3', doc))

100%|██████████| 120/120 [00:04<00:00, 27.00it/s]


In [5]:
preprocessing = [
    image.ForceResizeAug((224,224)),
    image.ColorNormalizeAug(mean=nd.array([0.485, 0.456, 0.406]), std=nd.array([0.229, 0.224, 0.225]))
]

def transform(data, label):
    data = data.astype('float32') / 255
    for pre in preprocessing:
        data = pre(data)
    
    data = nd.transpose(data, (2,0,1))
    return data, nd.array([label]).asscalar().astype('float32')

In [6]:
ctx = mx.gpu(0)
preprocessing[0] = image.ForceResizeAug((224,224))
imgs = vision.ImageFolderDataset('/home/samael/kaggle/dogs/data2/train2', transform=transform)
data = gluon.data.DataLoader(imgs, 64)

features_vgg_train, labels_train = get_features(models.vgg16_bn(pretrained=True, ctx=ctx), data)
features_resnet_train, _ = get_features(models.resnet152_v1(pretrained=True, ctx=ctx), data)
features_densenet_train, _ = get_features(models.densenet201(pretrained=True, ctx=ctx), data)

100%|██████████| 162/162 [02:54<00:00,  1.08s/it]


In [7]:
preprocessing[0] = image.ForceResizeAug((299,299))
imgs_299 = vision.ImageFolderDataset('/home/samael/kaggle/dogs/data2/train2', transform=transform)
data_299 = gluon.data.DataLoader(imgs_299, 64)

features_inception_train, _ = get_features(models.inception_v3(pretrained=True, ctx=ctx), data)

100%|██████████| 162/162 [01:45<00:00,  1.53it/s]


In [8]:
def accuracy(output, labels):
    return nd.mean(nd.argmax(output, axis=1) == labels).asscalar()

def evaluate(net, data_iter):
    loss, acc, n = 0., 0., 0.
    steps = len(data_iter)
    for data, label in data_iter:
        data, label = data.as_in_context(ctx), label.as_in_context(ctx)
        output = net(data)
        acc += accuracy(output, label)
        loss += nd.mean(softmax_cross_entropy(output, label)).asscalar()
    return loss/steps, acc/steps

In [9]:
with h5py.File('features_test.h5', 'r') as f:
    features_vgg_test = np.array(f['vgg'])
    features_resnet_test = np.array(f['resnet'])
    features_densenet_test = np.array(f['densenet'])
    features_inception_test = np.array(f['inception'])

In [10]:
features_resnet_train = features_resnet_train.reshape(features_resnet_train.shape[:2])
features_inception_train = features_inception_train.reshape(features_inception_train.shape[:2])

features_train = np.concatenate([features_resnet_train, features_densenet_train, features_inception_train], axis=-1)

In [11]:
X_train, X_val, y_train, y_val = train_test_split(features_train, labels_train, test_size=0.1)

# dataset_train = gluon.data.ArrayDataset(nd.array(X_train), nd.array(y_train))
dataset_train = gluon.data.ArrayDataset(nd.array(features_train), nd.array(labels_train))
dataset_val = gluon.data.ArrayDataset(nd.array(X_val), nd.array(y_val))

batch_size = 256
data_iter_train = gluon.data.DataLoader(dataset_train, batch_size, shuffle=True)
data_iter_val = gluon.data.DataLoader(dataset_val, batch_size)

In [12]:
X_train.shape

(9322, 6016)

In [13]:
ctx = mx.gpu(0)
net = nn.Sequential()
with net.name_scope():
    net.add(nn.BatchNorm())
    net.add(nn.Dense(1024))
    net.add(nn.BatchNorm())
    net.add(nn.Activation('relu'))
    net.add(nn.Dropout(0.5))
    net.add(nn.Dense(120))

net.initialize(ctx=ctx)
softmax_cross_entropy = gluon.loss.SoftmaxCrossEntropyLoss()
lr_sch = mx.lr_scheduler.FactorScheduler(step=1500, factor=0.5, stop_factor_lr=1e-11)
trainer = gluon.Trainer(net.collect_params(), 'adam', 
                        {'learning_rate': 1e-3, 'lr_scheduler': lr_sch})
net

Sequential(
  (0): BatchNorm(axis=1, eps=1e-05, momentum=0.9, fix_gamma=False, in_channels=None)
  (1): Dense(None -> 1024, linear)
  (2): BatchNorm(axis=1, eps=1e-05, momentum=0.9, fix_gamma=False, in_channels=None)
  (3): Activation(relu)
  (4): Dropout(p = 0.5)
  (5): Dense(None -> 120, linear)
)

In [14]:
from time import time
for epoch in range(100):
    start = time()
    train_loss = 0.
    train_acc = 0.
    steps = len(data_iter_train)
    for data, label in data_iter_train:
        data, label = data.as_in_context(ctx), label.as_in_context(ctx)
        
        with autograd.record():
            output = net(data)
            loss = softmax_cross_entropy(output, label)
        
        loss.backward()
        trainer.step(batch_size)
        
        train_loss += nd.mean(loss).asscalar()
        train_acc += accuracy(output, label)
    
    val_loss, val_acc = evaluate(net, data_iter_val)
    end = time() - start
    print("Epoch %d. loss: %.4f, acc: %.2f%%, val_loss %.4f, val_acc %.2f%%, times %.2fs, lr %s" % (
        epoch+1, train_loss/steps, train_acc/steps*100, val_loss, val_acc*100, end, str(trainer.learning_rate)))

Epoch 1. loss: 1.0348, acc: 75.11%, val_loss 0.1965, val_acc 93.96%, times 0.66s, lr 0.001
Epoch 2. loss: 0.2352, acc: 92.72%, val_loss 0.0939, val_acc 97.66%, times 0.58s, lr 0.001
Epoch 3. loss: 0.1452, acc: 95.58%, val_loss 0.0641, val_acc 98.28%, times 0.53s, lr 0.001
Epoch 4. loss: 0.1024, acc: 96.90%, val_loss 0.0390, val_acc 98.98%, times 0.54s, lr 0.001
Epoch 5. loss: 0.0655, acc: 98.31%, val_loss 0.0236, val_acc 99.77%, times 0.55s, lr 0.001
Epoch 6. loss: 0.0537, acc: 98.66%, val_loss 0.0176, val_acc 100.00%, times 0.64s, lr 0.001
Epoch 7. loss: 0.0405, acc: 99.14%, val_loss 0.0158, val_acc 99.77%, times 0.55s, lr 0.001
Epoch 8. loss: 0.0331, acc: 99.31%, val_loss 0.0104, val_acc 99.92%, times 0.49s, lr 0.001
Epoch 9. loss: 0.0264, acc: 99.42%, val_loss 0.0086, val_acc 100.00%, times 0.56s, lr 0.001
Epoch 10. loss: 0.0238, acc: 99.46%, val_loss 0.0071, val_acc 99.92%, times 0.59s, lr 0.001
Epoch 11. loss: 0.0167, acc: 99.69%, val_loss 0.0078, val_acc 99.84%, times 0.60s, lr 0

Epoch 89. loss: 0.0007, acc: 99.99%, val_loss 0.0000, val_acc 100.00%, times 0.57s, lr 0.00025
Epoch 90. loss: 0.0008, acc: 99.99%, val_loss 0.0000, val_acc 100.00%, times 0.57s, lr 0.00025
Epoch 91. loss: 0.0004, acc: 99.99%, val_loss 0.0000, val_acc 100.00%, times 0.61s, lr 0.00025
Epoch 92. loss: 0.0006, acc: 99.99%, val_loss 0.0000, val_acc 100.00%, times 0.58s, lr 0.00025
Epoch 93. loss: 0.0009, acc: 99.98%, val_loss 0.0000, val_acc 100.00%, times 0.54s, lr 0.00025
Epoch 94. loss: 0.0006, acc: 99.99%, val_loss 0.0000, val_acc 100.00%, times 0.53s, lr 0.00025
Epoch 95. loss: 0.0009, acc: 99.98%, val_loss 0.0000, val_acc 100.00%, times 0.54s, lr 0.00025
Epoch 96. loss: 0.0010, acc: 99.98%, val_loss 0.0000, val_acc 100.00%, times 0.51s, lr 0.00025
Epoch 97. loss: 0.0022, acc: 99.98%, val_loss 0.0000, val_acc 100.00%, times 0.58s, lr 0.00025
Epoch 98. loss: 0.0002, acc: 100.00%, val_loss 0.0000, val_acc 100.00%, times 0.66s, lr 0.00025
Epoch 99. loss: 0.0008, acc: 99.98%, val_loss 0.0

In [15]:
features_resnet_test = features_resnet_test.reshape(features_resnet_test.shape[:2])
features_inception_test = features_inception_test.reshape(features_inception_test.shape[:2])

features_test = np.concatenate([features_resnet_test, features_densenet_test, features_inception_test], axis=-1)

In [16]:
output = nd.softmax(net(nd.array(features_test).as_in_context(ctx))).asnumpy()

In [17]:
df = pd.read_csv('sample_submission.csv')

for i, c in enumerate(df.columns[1:]):
    df[c] = output[:,i]

df.to_csv('pred.csv', index=None)